In [1]:
from pathlib import Path
import os
from dotenv import load_dotenv
from tqdm import tqdm
tqdm.pandas()
%load_ext autoreload
%autoreload 2

In [2]:
# change dir to ml repo at ml/ root (if you started in notebook/)
os.chdir('../')
print(Path.cwd())
load_dotenv()

/home/opadmin/llama2d_opentable/lm_act_eval


True

In [3]:
from datasets import Dataset
import pandas as pd
from lm_act_eval.evaluation_harness.helper_functions.multion import (
  action_prefix,
  clean_extracted_text,
  extract_thought,
  extract_action,
  extract_explanation,
  ParseChatCompletion,
  extract_first
)

/home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Google Cloud not set up, skipping import of providers.gemini_utils.generate_from_gemini_completion
Google Cloud not set up, skipping import of vertexai.preview.generative_models.Image and llms.generate_from_gemini_completion


/home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/deepeval/__init__.py:41: UserWarning: You are using deepeval version 0.20.90, however version 0.21.16 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [4]:
from lm_act_eval._init_conf import init_conf
hydra_cfg = init_conf('trajectory_eval-experiment')
hydra_cfg

/home/opadmin/llama2d_opentable/lm_act_eval/lm_act_eval/_init_conf.py:9: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="../config")


{'sft': {'trajectory': {'data': {'path': 'lm_act_eval/.cache/five-star-trajectories/csv/data.csv', 'columns': {'y': 'ground_truth', "y_'": 'GPTV_generations'}, 'extract_fs': {'QUERY': {'QUERY': None}, 'screenshot': {'screenshot': None}, 'GOAL': {'chat_completion_messages': 'parse_completion.parse_content'}}, 'logging': {'project': 'opentable', 'result': 'lm_act_eval-run', 'hugging': {'space': 'multion-agi'}}}, 'comparator': {'gpt-v': {'model': 'gpt-4-vision-preview', 'max_token': 300, 'img_fidelity': 'high'}}, 'metrics': {'gpt-v': {'inputs': ['GOAL', 'QUERY', 'screenshots'], 'args': None}, 'llm_relevancy': ['explanation']}}}}

In [5]:
df = pd.read_csv(hydra_cfg.sft.trajectory.data.path)
print(
  df.chat_completion_messages.iloc[0][:100],
  df.columns
)

[{'role': 'system', 'content': '**Goal**: Let\'s play a game - You are an expert agent named MULTI·O Index(['session_id', 'idx_in_session', '_id', 'event_id', 'parent_id',
       'event_type', 'event_name', 'config', 'children_ids', 'inputs',
       'outputs', 'error', 'start_time', 'end_time', 'metadata', 'feedback',
       'metrics', 'user_properties', 'source', 'duration', 'project_id',
       'tenant', 'QUERY', 'URL', 'DOM', 'RULES', 'USER_CONTEXT',
       'PREV_ACTIONS', 'chat_history', 'CURRENT_TIME', 'planPrompt',
       'previousActionsRepetitionPrompt', 'chat_completion_messages',
       'ground_truth', 'screenshot', 'prompt', 'internal_metadata'],
      dtype='object')


In [30]:
from lm_act_eval.common.data.augmentations import *

In [27]:
usercontext_ = df.USER_CONTEXT.iloc[0:1]
print(usercontext_.apply(randomize_user_context).iloc[0])

('userAddress: 027 Savannah River, East Amy, SC 59069;\nuserEmail: morrisondawn@example.net;\nuserName: Nicholas Anderson;\nuserNotes: I am vegan, never order me any foods containing animal products.\n\nIf I tell you to buy something, get the purchase ready to execute, then check in with me about making the purchase.;', {'027 Savannah River, East Amy, SC 59069': '9926 Tesson Creek Estates Rd', 'morrisondawn@example.net': 'edmund.martin.mills@gmail.com', 'Nicholas Anderson': 'Edmund Mills'})


In [35]:
dom_example = df.DOM.iloc[0:1]
print(dom_example.apply(extract_and_replace_restaurant_names).iloc[0])

('<b id=0 aria-label="Next"/>\n<b id=1 aria-label="Toggle location picker"/>\n<t id=2>Find your table for any occasion/>\n<b id=3 aria-label="Date selector" html_id="search-autocomplete-day-picker" html_id="search-autocomplete-day-picker-label">Mar 16, 2024/>\n<s id=4 aria-label="Time selector">12:00 PM/>\n<t id=5>12:00 PM/>\n<s id=6 aria-label="Party size selector">2 people/>\n<t id=7>2 people/>\n<b id=8 role="combobox" html_id="home-page-autocomplete-label"/>\n<i id=9 placeholder="Location, Restaurant, or Cuisine" aria-label="Please input a Location, Restaurant or Cuisine" html_id="home-page-autocomplete-input"> />\n<b id=10 aria-label="Let’s go">Let’s go/>\n<l id=11 href="link_1" src="link_2" alt="A photo of Sawyer, Chapman and Erickson restaurant" title="" role="img" aria-label="4.4 stars">Promoted Sawyer, Chapman and Erickson 1950 reviews Steakhouse Price: Very Expensive St. Louis Booked 23 times today +1,000 pts +1,000 pts +1,000 pts/>\n<l id=12 href="link_3" aria-label="4:30 PM 

In [36]:
print(dom_example.apply(randomize_ids_links).iloc[0])

<b id=22 aria-label="Next"/>
<b id=31 aria-label="Toggle location picker"/>
<t id=24>Find your table for any occasion/>
<b id=23 aria-label="Date selector" html_id="search-autocomplete-day-picker" html_id="search-autocomplete-day-picker-label">Mar 16, 2024/>
<s id=3 aria-label="Time selector">12:00 PM/>
<t id=39>12:00 PM/>
<s id=22 aria-label="Party size selector">2 people/>
<t id=1>2 people/>
<b id=41 role="combobox" html_id="home-page-autocomplete-label"/>
<i id=1 placeholder="Location, Restaurant, or Cuisine" aria-label="Please input a Location, Restaurant or Cuisine" html_id="home-page-autocomplete-input"> />
<b id=42 aria-label="Let’s go">Let’s go/>
<l id=2 href="link_231" src="link_24" alt="A photo of Ruth's Chris Steak House - St. Louis restaurant" title="" role="img" aria-label="4.4 stars">Promoted Ruth's Chris Steak House - St. Louis 1950 reviews Steakhouse Price: Very Expensive St. Louis Booked 23 times today +1,000 pts +1,000 pts +1,000 pts/>
<l id=5 href="link_23" aria-labe

In [ ]:
from lm_act_eval.evaluation_harness.evaluators.sft.trajectory import TableTrajectoryEvaluator

# df_evaluator = TableTrajectoryEvaluator(hydra_cfg.sft.trajectory.data)

## Save/Load Data

In [ ]:
# traj_df_new = traj_df.merge(eligible_traj_df, how='left')
# traj_df_new.to_csv(dataset_path/'csv/data+gptv_eval.csv')

In [ ]:
table_name = 'data+gptv_eval-eligible'
artifact_name = 'gptv_opentable-trajectory_eval'
eligible_traj_df.to_csv(dataset_path/f'csv/{table_name}.csv')

In [ ]:
eligible_traj_df = pd.read_csv(
  dataset_path/f'csv/{table_name}.csv', index_col=0)
eligible_traj_df.index.name = None
eligible_traj_df.head(2)

,session_id,idx_in_session,_id,event_id,parent_id,event_type,event_name,config,children_ids,inputs,...,planPrompt,previousActionsRepetitionPrompt,chat_completion_messages,ground_truth,screenshot,prompt,internal_metadata,GPTV response,chat_completion_messages_content,GPTV_evaluation
169,36c2f0ba-e215-4081-96b4-ecb04c10a517,0,65ed4fc8166e07fe3c29c476,6c8932f3-c31d-4683-aeb0-52a5bbe5a978,36c2f0ba-e215-4081-96b4-ecb04c10a517,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'What was the actual enrollment coun...,...,"IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",PLAN:\n1. Use the Google search function to fi...,https://multion-client-screenshots.s3.us-east-...,NaN,NaN,It appears that you are looking for informatio...,**Goal**: Let's play a game - You are an exper...,SCORE: 60\nEXPLANATION:\nThe user appears to b...
170,36c2f0ba-e215-4081-96b4-ecb04c10a517,1,65ed4fd1b2b5b0c1673c47d7,172ff25e-efdb-4002-b43a-0180f3f0eb19,36c2f0ba-e215-4081-96b4-ecb04c10a517,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'What was the actual enrollment coun...,...,"IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",COMMANDS:\nCLICK 26\nEXPLANATION: I am clickin...,https://multion-client-screenshots.s3.us-east-...,NaN,NaN,The image shows a webpage from ClinicalTrials....,**Goal**: Let's play a game - You are an exper...,SCORE: 95\nEXPLANATION:\nThe user is very clos...


Logging

In [ ]:
# log data
import wandb
from dotenv import load_dotenv
load_dotenv()
wandb.login(relogin=True)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/opadmin/.netrc


True

In [ ]:
opentable_artifact = wandb.Artifact("opentable_trajectories-gptv_eval", type="dataset")
# opentable_table = wandb.Table(dataframe=traj_df_new)
opentable_table_eligible = wandb.Table(dataframe=eligible_traj_df)
#
# opentable_artifact.add(opentable_table, "opentable")
opentable_artifact.add(
  opentable_table_eligible, artifact_name)
# opentable_artifact.add_file(str(dataset_path/'csv/data+gptv.csv'))
opentable_artifact.add_file(str(dataset_path/f'csv/{table_name}.csv'))


In [ ]:
# Start a W&B run to log data
run = wandb.init(project="trajectory_eval", reinit=True)
# Log the table to visualize with a run...
run.log({"opentable_gptv": opentable_table_eligible})

# and Log as an Artifact to increase the available row limit!
run.log_artifact(opentable_artifact)

wandb: Currently logged in as: mmym (multion-agi). Use `wandb login --relogin` to force relogin


wandb: WARNING Serializing object of type str that is 531752 bytes
wandb: WARNING Serializing object of type str that is 517908 bytes
wandb: WARNING Serializing object of type str that is 541284 bytes
wandb: WARNING Serializing object of type str that is 525810 bytes
wandb: WARNING Serializing object of type str that is 116864 bytes
wandb: WARNING Serializing object of type str that is 121274 bytes
wandb: WARNING Serializing object of type str that is 120688 bytes
wandb: WARNING Serializing object of type str that is 124364 bytes
wandb: WARNING Serializing object of type str that is 127354 bytes
wandb: WARNING Serializing object of type str that is 129308 bytes
wandb: WARNING Serializing object of type str that is 129582 bytes
wandb: WARNING Serializing object of type str that is 135310 bytes
wandb: WARNING Serializing object of type str that is 137856 bytes
wandb: WARNING Serializing object of type str that is 137644 bytes
wandb: WARNING Serializing object of type str that is 136800 b

<Artifact opentable_trajectories-gptv_eval>